In [ ]:
import importlib
importlib.reload(main)

In [1]:
# onnx to pt
import onnx
from onnx2pytorch import ConvertModel
import torch
import numpy as np

# 1) 加载 onnx 模型
onnx_model = onnx.load("model/armor.onnx")

# 可选：检查 onnx 是否有效
import onnx.checker
onnx.checker.check_model(onnx_model)  # 若报错说明 onnx 本身有问题

# 2) 转换为 PyTorch nn.Module
pytorch_model = ConvertModel(onnx_model)

# 3) 测试一次 forward（确保输入 shape/类型正确）
# 这里假设模型期望输入是 [1,3,H,W]，H/W 换成你模型实际的尺寸（例如 640）
# dummy = torch.randn(1, 3, 640, 640)  # 根据你的模型输入改形状
# with torch.no_grad():
#     out = pytorch_model(dummy)
# print("forward output shape:", type(out), getattr(out, "shape", None))

# 4) 保存模型 —— 两种常用方法：
# A) 保存 whole module（序列化完整对象 — 以后 torch.load 可以直接恢复）
torch.save(pytorch_model, "model/armor.pt")

# B) 或者保存 state_dict（更常见、跨环境更稳妥）
# torch.save(pytorch_model.state_dict(), "model/armor_state_dict.pt")
# 若保存 state_dict，加载时需要重新 ConvertModel(onnx_model) 再 load_state_dict

# 示范如何用 state_dict 加载（之后在另一个脚本里）：
# model = ConvertModel(onnx.load("model/armor.onnx"))
# model.load_state_dict(torch.load("model/armor_state_dict.pt"))
# model.eval()


e:\software\anaconda\envs\roboflow\lib\site-packages\onnx2pytorch\convert\layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:209.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
e:\software\anaconda\envs\roboflow\lib\site-packages\onnx2pytorch\convert\attribute.py:106: UserWarning: Pytorch's interpolate uses no coordinate_transformation_mode=asymmetric. Result might differ.
  warnings.warn(
e:\software\anaconda\envs\roboflow\lib\site-packages\onnx2pytorch\operations\resize.py:16: UserWarning: Pytorch's interpolate uses no cubic_coeff_a. Result might diffe

In [2]:
# load the model to Roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="rxgD3vU1PAkQz73bYYtN")
project = rf.workspace().project("rmcars_test-eena7")

#can specify weights_filename, default is "weights/best.pt"
version = project.version(1)

#example1 - directory path is "training1/model1.pt" for yolov8 model
version.deploy("yolov8", "model", "armor.pt")

#example2 - directory path is "training1/weights/best.pt" for yolov8 model
# version.deploy("yolov8m", "training1")

loading Roboflow workspace...
loading Roboflow project...


TypeError: argument of type 'ConvertModel' is not iterable